In [1]:
import pandas as pd
df = pd.read_csv("news_data.csv")
print(df.columns)
print(df.head(3))

Index(['title', 'subtitle', 'author', 'url', 'text', 'time', 'date',
       'difficulty', 'tags'],
      dtype='object')
                                               title subtitle  \
0  Астрономы нанесли на карту 25 тысяч сверхмасси...      NaN   
1  Астрономы нашли пять гигантских собратьев Эты ...      NaN   
2  Глицерин из электронных сигарет спровоцировал ...      NaN   

             author                                            url  \
0  Александр Войтюк    https://nplus1.ru/news/2021/02/24/map-of-bh   
1  Владимир Королев  https://nplus1.ru/news/2016/01/07/eta-carinae   
2     Слава Гоменюк    https://nplus1.ru/news/2020/05/27/microends   

                                                text   time      date  \
0  Астрономы подвели предварительные итоги низкоч...  10:23  24.02.21   
1  Астрономы из Университета Огайо и Центра косми...  12:10  07.01.16   
2  Глицерол и пропиленгликоль, входящие в состав ...  21:00  27.05.20   

   difficulty                               

In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
index_name = "news"


In [3]:
mapping = {
    "settings": {
        "analysis": {
            "filter": {
                "russian_stop": {
                    "type": "stop",
                    "stopwords": "_russian_"  # стандартные русские стоп-слова
                },
                "russian_stemmer": {
                    "type": "stemmer",
                    "language": "russian"
                }
            },
            "analyzer": {
                "russian_custom": {
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "russian_stop",
                        "russian_stemmer"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {"type": "text", "analyzer": "russian_custom"},
            "subtitle": {"type": "text", "analyzer": "russian_custom"},
            "author": {"type": "keyword"},
            "url": {"type": "keyword", "index": False},
            "text": {"type": "text", "analyzer": "russian_custom"},
            "time": {"type": "keyword"},
            "date": {"type": "date", "format": "dd.MM.yy||yyyy-MM-dd"},
            "difficulty": {"type": "float"},
            "tags": {"type": "keyword"},
            "true_title": {"type": "text", "index": False}
        }
    }
}

es.indices.delete(index=index_name, ignore_unavailable=True)
es.indices.create(index=index_name, body=mapping)
print(f"Индекс {index_name} успешно создан с русским анализатором")

Индекс news успешно создан с русским анализатором


In [4]:
import csv
from elasticsearch import Elasticsearch
from normalize_utils import (
    normalize_date,
    normalize_difficulty,
    normalize_tags,
    lemmatize_text,
    clean_text
)

es = Elasticsearch("http://localhost:9200")
index_name = "news"

with open("news_data.csv", newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        row["date"] = normalize_date(row.get("date", ""))
        row["difficulty"] = normalize_difficulty(row.get("difficulty"))
        row["tags"] = normalize_tags(row.get("tags"))

        # Лемматизируем большие текстовые поля
        for field in ["title", "subtitle", "text"]:
            if row.get(field):
                row[field] = lemmatize_text(clean_text(row[field]))

        es.index(index=index_name, document=row)

print("CSV-файл успешно предобработан и загружен в Elasticsearch")


CSV-файл успешно предобработан и загружен в Elasticsearch
